In [1]:
import os
os.chdir('..')

In [2]:
# %%capture
# !pip install --upgrade wandb
# !wandb login 

In [15]:
import wandb
from transformers import BertTokenizer, BertModel, GPT2Model, GPT2Tokenizer

In [16]:
tokenization = 'gpt2'
pretrained_weights = 'gpt2'

tokenizer = GPT2Tokenizer.from_pretrained(pretrained_weights, unk_token='<unk>', eos_token='<pad>', pad_token='<pad>', bos_token='<start>')
# pad_index = tokenizer.convert_tokens_to_ids('<pad>')

In [37]:
vocab = tokenizer.get_vocab()

In [18]:
print(tokenizer.special_tokens_map)
# tokenizer.bos_token = '<start>'
# tokenizer.eos_token = '<pad>'
# tokenizer.pad_token = '<pad>'
# tokenizer.unk_token = '<unk>'
# tokenizer.special_tokens_map

{'bos_token': '<start>', 'eos_token': '<pad>', 'unk_token': '<unk>', 'pad_token': '<pad>'}


In [20]:
vocab['<start>']

KeyError: '<start>'

In [34]:
vocab['<unk>']

KeyError: '<unk>'

In [45]:
vocab['<unk>']

'50259'

In [44]:
vocab = {key: str(value) for key,value in vocab.items()}

In [36]:
len(vocab)

50260

In [74]:
vocab_inverse = {str(value):key for key,value in vocab.items()}

In [76]:
vocab_inverse['50259']

'<unk>'

In [77]:
with open('data/wi_gpt.txt', 'w', encoding='utf-8') as dictout:
    dictout.write(str(vocab))
with open('data/iw_gpt.txt', 'w', encoding='utf-8') as dictout:
    dictout.write(str(vocab_inverse))
print('total tokens: ', len(vocab))

total tokens:  50260


In [3]:
from utils.preprocess import load_dict

In [4]:
path = r"data"
w2i, i2w = load_dict(path)

In [5]:
len(w2i), len(i2w)

(50260, 50260)

In [16]:
d = tokenizer.encode('I love dogs and tis isidb; ;ljg;xj ;lkgj;')
d

[40,
 1842,
 6844,
 290,
 256,
 271,
 318,
 312,
 65,
 26,
 2162,
 75,
 73,
 70,
 26,
 87,
 73,
 2162,
 75,
 10025,
 73,
 26]

In [35]:
tr = tokenizer.tokenize('I love dogs and tis isidb; ;ljg;xj ;lkgj;')
tr

['I',
 'Ġlove',
 'Ġdogs',
 'Ġand',
 'Ġt',
 'is',
 'Ġis',
 'id',
 'b',
 ';',
 'Ġ;',
 'l',
 'j',
 'g',
 ';',
 'x',
 'j',
 'Ġ;',
 'l',
 'kg',
 'j',
 ';']

In [38]:
vocab['Ġlove']

1842

In [17]:
e = [tokenizer.decode(x) for x in d]
e

['I',
 ' love',
 ' dogs',
 ' and',
 ' t',
 'is',
 ' is',
 'id',
 'b',
 ';',
 ' ;',
 'l',
 'j',
 'g',
 ';',
 'x',
 'j',
 ' ;',
 'l',
 'kg',
 'j',
 ';']

In [18]:
tokenizer.decode(d)

'I love dogs and tis isidb; ;ljg;xj ;lkgj;'

In [15]:
from utils.preprocess import get_tokenized

In [16]:
tokenized = get_tokenized(path)

In [17]:
len(tokenized)

20000

In [18]:
tokenized[-10]

['(',
 'Po',
 'ets',
 '&',
 'Qu',
 'ants',
 ')',
 'ĠâĢĶ',
 'ĠDavid',
 'ĠThomas',
 'Ġdigs',
 'ĠB',
 'H',
 'AG',
 's',
 'ĠâĢĵ',
 'Ġshort',
 'Ġfor',
 'Ġbig',
 ',',
 'Ġhairy',
 ',',
 'Ġaud',
 'acious',
 'Ġgoals',
 '.']

In [6]:
from utils.dataloader import GANDataset, GenDataIter
BATCH_SIZE = 64

In [7]:
iterator = GenDataIter(data=path,
                            batch_size=BATCH_SIZE,
                            shuffle=None)

In [8]:
loader = iterator.loader
loader

In [9]:
for el in loader:
    print(el)
    x = el['input']
    
    for token in x:
        print([iterator.idx2word[str(l.item())] for l in token])
        break
    break

{'input': tensor([[    1, 39461,    82,  ..., 17168,    13,     0],
        [    1,  2990,  8020,  ...,     0,     0,     0],
        [    1,  1532,   345,  ...,     0,     0,     0],
        ...,
        [    1, 29316,  6705,  ...,   905,   309, 10008],
        [    1, 29387, 16988,  ...,     0,     0,     0],
        [    1,  1544,   531,  ...,     0,     0,     0]]), 'target': tensor([[39461,    82,  4186,  ...,    13,     0,     0],
        [ 2990,  8020,   470,  ...,     0,     0,     0],
        [ 1532,   345,   447,  ...,     0,     0,     0],
        ...,
        [29316,  6705,  2060,  ...,   309, 10008,    25],
        [29387, 16988,  1139,  ...,     0,     0,     0],
        [ 1544,   531,   262,  ...,     0,     0,     0]])}
['"', 'Brother', 's', 'ĠTom', ',', 'Ġleft', ',', 'Ġand', 'ĠRay', 'ĠMag', 'lio', 'zzi', ',', 'Ġshown', 'Ġin', 'Ġ1991', ',', 'Ġsay', 'Ġthey', "'ll", 'âĢ¦', 'Ġ(', 'Susan', 'ĠWalsh', ',', 'ĠAssociated', 'âĢ¦)', 'ĠThey', 'Ġwere', 'Ġa', 'Ġcouple', 'Ġof', 'Ġaut

In [10]:
iterator.word2idx['<pad>']

'50258'

In [28]:
len(iterator.word2idx)

50260

In [11]:
iterator.idx2word['1'], iterator.idx2word['9930'], iterator.idx2word['18108'], iterator.idx2word['0']

('"', 'they', 'had', '!')

In [12]:
iterator.word2idx['<start>']

'50257'

In [24]:
from utils.preprocess import tokens_to_tensor, tensor_to_tokens

In [19]:
t = tokenized[0]
t

['Brother',
 's',
 'ĠTom',
 ',',
 'Ġleft',
 ',',
 'Ġand',
 'ĠRay',
 'ĠMag',
 'lio',
 'zzi',
 ',',
 'Ġshown',
 'Ġin',
 'Ġ1991',
 ',',
 'Ġsay',
 'Ġthey',
 "'ll",
 'âĢ¦',
 'Ġ(',
 'Susan',
 'ĠWalsh',
 ',',
 'ĠAssociated',
 'âĢ¦)',
 'ĠThey',
 'Ġwere',
 'Ġa',
 'Ġcouple',
 'Ġof',
 'Ġauto',
 'Ġmechanics',
 'Ġwith',
 'Ġa',
 'Ġpronounced',
 'ĠBoston',
 'Ġbro',
 'gue',
 'Ġand',
 ',',
 'Ġimpro',
 'bably',
 ',',
 'Ġdegrees',
 'Ġfrom',
 'ĠMIT',
 '.']

In [25]:
tokens_to_tensor([t], w2i)

tensor([[39461,    82,  4186,    11,  1364,    11,   290,  7760,  2944, 48590,
         46218,    11,  3402,   287, 10249,    11,   910,   484,  1183,  1399,
           357, 45842, 24104,    11, 10575, 38418,  1119,   547,   257,  3155,
           286,  8295, 12933,   351,   257, 16293,  6182,  1379, 18701,   290,
            11,  2015, 11921,    11,  7370,   422, 17168,    13,     0,     0]])

In [21]:
i2w['11']

','

In [22]:
i2w['0']

'!'

In [29]:
import torch
from models.RelGAN_G import RelGAN_G
import configuration as cfg
MAX_SEQ_LEN = cfg.MAX_SEQ_LEN
PAD_IDX = cfg.PAD_IDX
VOCAB_SIZE = 50260
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if_cuda = True if torch.cuda.is_available() else False

In [30]:
gen = RelGAN_G(mem_slots=1, num_heads=2, head_size=256, embedding_dim=32, hidden_dim=32,
                            vocab_size=VOCAB_SIZE, max_seq_len=MAX_SEQ_LEN, padding_idx=PAD_IDX, gpu=if_cuda)

In [ ]:
from models.RelGAN_D import RelGAN_D
dis = RelGAN_D(embed_dim=64, 
               max_seq_len=MAX_SEQ_LEN,
               num_rep=64, 
               vocab_size=VOCAB_SIZE, 
               padding_idx=PAD_IDX, 
               weights = 'uniform',
               gpu=if_cuda, 
               dropout=0.25)

In [33]:
model = GPT2Model.from_pretrained('gpt2')
embeddings_pretrained = model.get_input_embeddings()

embeddings_pretrained

Embedding(50257, 768)

In [35]:
embeddings_pretrained.weight.data.size()

torch.Size([50257, 768])

In [36]:
from models.LSTM_D import LSTM_D

In [38]:
# ef __init__(self, vocab_size, embed_dim, hidden_size,
      #           emb_pretrained, embeddings, max_seq_len, padding_idx, weights, gpu=cfg.if_cuda):
    
dis = LSTM_D(vocab_size=50260,
            embed_dim=768,
            hidden_size=256,
            emb_pretrained = False,
            embeddings = embeddings_pretrained,
            max_seq_len=cfg.MAX_SEQ_LEN,
            padding_idx=cfg.PAD_IDX,
            weights='uniform')

In [2]:
from models.instructor_LSTM import LSTMInstructor
instructor = LSTMInstructor()

In [3]:
len(instructor.word2idx_dict), len(instructor.idx2word_dict)

(50260, 50260)

In [4]:
instructor.adv_train_generator(1)

0.6880250573158264

In [5]:
instructor.adv_train_discriminator(1)

0.6984397768974304

In [6]:
instructor._run()

Starting Adversarial Training


g_loss: 0.6940, d_loss: 0.6815, temperature: 1.0000:   0%| | 0/100 [14:35<?, ?i

KeyboardInterrupt: 

In [ ]:
# convert_ids_to_tokens
# decode

In [7]:
# from utils.dataloader import GANDataset, GenDataIter
# from utils.preprocess import *
SAMPLES_NUM = 10
BATCH_SIZE = 10

eval_samples = instructor.gen.sample(SAMPLES_NUM, 4 * BATCH_SIZE)

In [26]:
from utils.preprocess import tensor_to_tokens
gen_tokens = tensor_to_tokens(eval_samples, instructor.idx2word_dict)

In [32]:
def save_tokens_transformer(filename, samples):
    if cfg.tokenizator == 'GPT2':
        tokenizer = GPT2Tokenizer.from_pretrained('gpt2', unk_token='<unk>', eos_token='<pad>',
                                                   pad_token='<pad>', bos_token='<start>')
    elif cfg.tokenizator == 'BERT':  # TODO implement
        raise NotImplementedError
    
    texts = [tokenizer.decode(x) for x in samples]
    with open(filename, 'w', encoding='utf-8') as f:
        for sent in texts:
            f.write(sent)
            f.write('\n')

In [33]:
path = os.getcwd()+'dddd.txt'

In [34]:
import os
import configuration as cfg
save_tokens_transformer(path, eval_samples)

In [10]:
for el in gen_tokens:
    print(el)
    print()

['Ġoptimize', 'ĠLon', 'Cast', 'ĠSchiff', '--------------', 'boa', 'Ġbystanders', 'Ġnascent', 'Ġ503', 'Ġrental', 'Ġstarved', 'ĠReact', 'ĠSwe', 'Ġcontain', 'ĠFront', 'Ġmyst', 'ĠUzbek', 'ĠESA', 'mL', 'pers', 'Ġstandby', 'ĠSchn', 'ĠShotgun', 'ĠArsenal', 'Ġdonated', 'ĠSteps', 'Ġblessings', 'sounding', 'Ġsaving', 'lection', 'Ġnations', 'Human', 'Ġripping', 'Ġimportance', 'ĠSingle', 'Ġemission', 'False', 'ij', 'containing', 'acio', 'ĠEnc', '591', 'ĠBreast', 'hedon', 'tc', 'axe', 'iatrics', 'IRED', 'rider', 'ĠCalling']

['Ġstall', 'Ġeru', 'ĠCly', 'Ġmelted', "')", 'Ġitching', 'Ġwarranties', 'éĩ', 'ĠExercise', 'Ġconditioning', 'ĠBaron', 'ĠGohan', 'ĠGREAT', 'verbs', 'ĠConst', 'ĠVT', 'Ġabandon', 'Ġindicated', 'ĠBars', 'Syn', 'ĠParent', 'arist', 'Ġconverting', 'ocaly', 'ĠHistorically', 'Ġblasts', 'Ġcapitalize', 'Ġgenerators', 'Ġnotwithstanding', 'Ġ:=', 'Ġnodd', 'ĠStay', 'Ġdrinkers', 'ĠBerg', 'ousing', 'Ġmemories', 'Ġcontrolled', 'Ġ342', 'rush', 'Ġbranding', 'ĠSPL', 'AGE', 'Ġcollegiate', 'Ġcompan', 